In [4]:
import pandas as pd
import numpy as np
import pymysql

In [96]:
text = 'train.txt'

In [98]:
###### 기본적인 전처리 리스트 ######
# 텍스트 파일에서 필요없는 데이터의 공통분모를 조건으로 제거
# lines, a와 같은 변수는 함수를 사용하는 지점에서 미리 선언하면 문제는 없었음

# 앞부분부터 cjs로 시작하는 내용을 전부 제거
def fir():
    file = open(text, 'w', encoding='UTF8')
    for i in lines:
        if i[:3] == 'Web':
            file.write(i.replace(a, ''))
        else:
            file.write(i)
    file.close()

# 앞부분부터 공백이 아니게 된 것들에 전체 공백을 줌 (사용된 텍스트파일의 특성반영)
def seco():
    file = open(text, 'w', encoding='UTF8')
    for i in lines:
        if i[:2] != ' ':
            file.write(i.replace(i[:2], ' '))
        else:
            file.write(i)
    file.close()

# 전체 파일에 좌측 여백 제거 (지금 체크결과 2번과 3번함수는 3번만 사용해도 될듯함)
def thi():
    file = open(text, 'w', encoding='UTF8')
    for i in lines:
        file.write(i.lstrip())
    file.close()

# 필요한 줄의 특성을 체크하여 if문으로 걸러냄
def fo():
    file = open(text, 'w', encoding='UTF8')
    for i in lines:
        if i[:4] == '시간 2':
            file.write(i)
        elif i[:4] == 'mmsi':
            file.write(i)
        elif i[:4] == 'ais_':
            file.write(i)
        elif i[:4] == 'cog ':
            file.write(i)
        elif i[:4] == 'sog ':
            file.write(i)
        elif i[:4] == 'ship':
            file.write(i)
    file.close()

# 선박 명 수정, 특정 선박의 이름에 문제가 발견되어서 생성
def Na():
    file = open(text, 'w', encoding='UTF8')
    file.write('\n')
    for i in lines:
        if i[:8] == 'shipName':
            file.write('shipName'+' '+i[9:].replace(',', '.'))
        else:
            file.write(i)
    file.close()

##### CSV전환 #####
def proc():
    newF = 'test.csv'
    count = 0       # 리스트화 하여 나누기위한 기준
    txtF = open(newF, 'a', encoding='cp949')
    txtF.write("MMSI,degree,velocity,name,type,longitude,latitude,time")
    for line in lines :
        line.replace("\n", "")  # 개행을 전부 제거 : ','를 사용하여 정렬
        index = line.find(' ')  # find()함수를 통해 ' '를 마킹
        line = line.replace(" ", "")    # 공백을 제거
        line = line[index: ]    # 공백뒷자리(결과값)만 정렬
      
        if count % 8 != 0 :
            line = line.replace("\n", ", ") # 개행부분은 전부 ','로 대체
        count += 1                      # 데이터 개수(9개)를 초과여부 분별
        txtF.write(line)            # 데이터 출력
    txtF.close()

In [20]:
# 파일리딩 및 1차커팅
file = open(text, 'r', encoding='UTF8')
lines = file.readlines()
file.close()
a = 'WebSocket.vue?c0fd:'
fir()

In [21]:
# 2차 커팅
file = open(text, 'r', encoding='UTF8')
lines = file.readlines()
file.close()
seco()

In [22]:
# 3차 커팅
file = open(text, 'r', encoding='UTF8')
lines = file.readlines()
file.close()
thi()

In [23]:
# 파일에 시간표기가 안되어있을 때 
file = open(text, 'r', encoding='utf8')
lines = file.readlines()
file.close()

file = open(text, 'w', encoding='utf8')
for i in lines:
    if i[:2] == '20':
        file.write('시간'+' '+i)
    else:
        file.write(i)
file.close()

In [24]:
# 파일에서 필요한 데이터 추출 : 시간, mmsi, posX, posY, cog, sog, shipName, shipType
file = open(text, 'r', encoding='UTF8')
lines = file.readlines()
file.close()
fo()

In [25]:
# 선박 명 부분 고칠 방법 찾기
file = open(text, 'r', encoding='UTF8')
lines = file.readlines()
file.close()
Na()

In [109]:
file = open(text, 'r', encoding='UTF8')
# txt파일 첫줄 비어있어야 작동함
lines = file.readlines()
file.close()
proc()

## 훈련데이터 프레임

In [2]:
test = 'test.csv'

In [5]:
df = pd.read_csv(test, encoding='cp949')

In [4]:
df['type'].unique()

array([' 80', ' 30', ' 72', ' 37', ' 52', ' 89', ' 54', ' 70', ' 0',
       ' 50', ' ', ' 10', ' 71', ' 51', ' 31', ' 60', ' 33', ' 32', ' 90',
       ' 99', ' 36', ' 55', ' 53', ' 73', ' 79', ' 40', ' 98', ' 35',
       ' 34', ' 74', ' 91'], dtype=object)

In [ ]:
df['name'].unique()

In [ ]:
for i in range(len(df['type'])):
    df['type'][i] = df['type'][i].replace(' ', '')
    df['degree'][i] = df['degree'][i].replace(' ', '')
    df['name'][i] = df['name'][i].replace(' ', '')

In [12]:
df01 = df[df['type'] == '80']
df02 = df[df['type'] == '70']

In [40]:
len(df01['latitude'].unique())

2562

## 기상데이터프레임

In [ ]:
check = pd.read_csv('해양.csv', encoding='cp949')
check.head()

In [25]:
check = check[['관측소명', '관측시간', '유의파고', '유의파주기', '파향', '유속(Kn & cm/s)', '유향(˚)', '풍속(m/s)', '풍향']]
check.columns = ['관측소', '시간', '파고', '파주기', '파향', '유속(Kn)', '유향', '풍속(m/s)', '풍향']

In [26]:
check.to_csv('check.csv')

## DB연결

In [52]:
conn = pymysql.connect(host='localhost', user='root', password='taldalim', db='world', charset='utf8')
# connect(host = '서버주소', user = 'username', password = 'password', db = 'dbname', charset = 'utf8')

In [ ]:
# CREATE
try:
    curs = conn.cursor()
    sql = '''CREATE TABLE 테이블명 (
        id int(11) NOT NULL AUTO_INCREMENT PRIMARY KEY,
        name varchar(255),
        email varchar(255)
        )
        '''
    curs.execute(sql)
    conn.commit()

finally:
    conn.close()

In [ ]:
# INSERT 1
try:
    curs = conn.cursor()
    sql = "INSERT INTO user VALUES (%s, %s, %s)"
    val = (1111, "kim", "google")
    curs.execute(sql, val)
    conn.commit()

finally:
    conn.close()

In [ ]:
# INSERT 2 / df스타일
for idx, row in df.iterrows():
    curs = conn.cursor()
    sql = "INSERT INTO user VALUES (%s)"
    val = (row.승객, row.이름, row.성별)
    curs.execute(sql, val)
conn.commit()
conn.close()

In [ ]:
# SELECT
try:
    curs = conn.cursor()
    sql = "SELECT * FROM user"
    curs.execute(sql)
    result = curs.fetchall()
    conn.commit()

    for i in result:
        print(i)

finally:
    conn.close()

In [ ]:
# SELECT(특정 데이터 확인)
try:
    curs = conn.cursor()
    sql = "SELECT * FROM user WHERE name=%s"
    val = ("Kim")
    curs.execute(sql, val)
    result = curs.fetchall()
    conn.commit()

    for i in result:
        print(i)

finally:
    conn.close()

In [ ]:
# UPDATE
try:
    curs = conn.cursor()
    sql = "UPDATE user SET name=%s WHERE name=%s"
    val = ("Kim", "kim")
    curs.execute(sql, val)
    conn.commit()

finally:
    conn.close()

In [ ]:
# DELETE
try:
    curs = conn.cursor()
    sql = "DELETE FROM user WHERE name=%s"
    val = ("Seo")
    curs.execute(sql, val)
    conn.commit()

finally:
    conn.close()